In [860]:
!pip install pypdf2

In [861]:
!pip install unidecode

In [862]:
# Descargamos el paquete de la libreria spacy
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 20.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [863]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import PyPDF2
import unicodedata
import nltk
import es_core_news_sm
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


In [864]:
with open('/content/tableros.yvera.tur.ar_tablero_ODS_.html', 'r') as archivo:
    # Lee todo el contenido del archivo
    contenido_html = archivo.read()

# Crea un objeto BeautifulSoup
soup = BeautifulSoup(contenido_html, 'lxml')

# Usa el método .text para obtener solo el texto, sin las etiquetas HTML
texto_plano = soup.find('div', {'title' : "Metodología"}).text

# Imprime el texto plano
print(texto_plano)






Definiciones y conceptos

El tablero presenta un conjunto de indicadores que buscan reflejar el aporte del 
                                turismo a los Objetivos de Desarrollo Sostenible (ODS) para nuestro país. El Documento de Trabajo 
                                N°12: El turismo en los Objetivos de Desarrollo Sostenible aborda esta propuesta y describe en detalle las fuentas utilizadas y los indicadores elaborados
Fuentes de información


                          Los indicadores que reflejan el aporte del turismo a la economía en su conjunto en términos de actividad 
                                económica y empleo, y la tasa de informalidad de éste, provienen de la 
                          Cuenta Satélite de Turismo de Argentina (CST-A)
                          . El resto de los indicadores, que buscan medir el acceso al turismo y su nivel de desigualdad,
                                toman como fuente de los datos a la 
                          Encuesta de Viaje

In [865]:
# Abre el archivo en modo binario de lectura ('rb')
with open('/content/PROYECTO DE LEY TURISMO SOCIAL 2004.pdf', 'rb') as archivo:
    # Crea un objeto PdfFileReader
    lector = PyPDF2.PdfReader(archivo)

    # Inicializa una cadena vacía para almacenar el texto
    texto = ''

    # Itera sobre todas las páginas del PDF
    for i in range(len(lector.pages)):
        # Obtiene la página
        pagina = lector.pages[i]

        # Extrae el texto de la página y lo añade a la cadena de texto
        texto += pagina.extract_text()

# Imprime el texto extraído
print(texto)

Ley Nacional de Turismo Social 
Título 1 
Turismo Social 
Capitulo 1 
Disposiciones generales 
ARTICULO l.- Turismo social. El estado garantizará a los habitantes de 
recursos económicos limitados, el derecho al descanso y al tiempo libre, 
permitiéndoles acceder a las distintas zonas turísticas, afianzando el 
sentimiento de pertenencia. 
El turismo social comprende todos aquellos instrumentos y medios que 
otorguen facilidades para que personas de recursos económicos limitados, o 
con capacidades dl$zrentes, accedan a viajes turísticos con fines recreativos en 
condiciones adecuadas de economía, seguridad y comodidad. 
ARTICULO 2: Finalidad. La presente ley tiene por finalidad 
4 Elaborar un plan estratégico nacional de turismo social “PENTS” 
b) Promover el desarrollo y mejoramiento del turismo social; 
c) Fomentar el ejercicio del turismo social, conservando y preservando el 
patrimonio natural, histórico y cultural de nuestro país. Elaborando en cada 
caso los estudios de impacto 

In [866]:
def convertir_a_minusculas(texto):
  tokens = word_tokenize(texto)
  tokens = [token.lower() for token in tokens]  # Convertir a minúsculas
  return ' '.join(tokens)

In [867]:
# Función de eliminación de acentos
def remueve_acentos(texto):
    nfkd_form = unicodedata.normalize('NFKD', texto)
    return''.join([letra for letra in nfkd_form if not unicodedata.combining(letra)])

In [868]:
def remueve_signos_puntuacion(texto):
    texto = re.sub(r'[^\w\s]','',texto)
    return texto

In [869]:
# Funcion para eliminar las palabras de parada
def remueve_palabras(texto):
  palabra_tokens = word_tokenize(texto)
  texto_filtrado = [palabra for palabra in palabra_tokens \
                    if palabra.casefold() not in palabra_parada]
  return ' '.join(texto_filtrado)

In [870]:
# Descargar el conjunto de palabras de parada en español
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [871]:
# Elegir las palabras de parada para idioma español
palabra_parada = set(stopwords.words('spanish'))

In [872]:
# Creamos el objeto
nlp = es_core_news_sm.load()

In [873]:
texto_plano = remueve_palabras(texto_plano)

In [874]:
texto_plano = convertir_a_minusculas(texto_plano)

In [875]:
texto_plano = remueve_acentos(texto_plano)

In [876]:
texto_plano = texto_plano.split(".")
texto_plano

['definiciones conceptos tablero presenta conjunto indicadores buscan reflejar aporte turismo objetivos desarrollo sostenible ( ods ) pais ',
 ' documento trabajo n°12 : turismo objetivos desarrollo sostenible aborda propuesta describe detalle fuentas utilizadas indicadores elaborados fuentes informacion indicadores reflejan aporte turismo economia conjunto terminos actividad economica empleo , tasa informalidad este , provienen cuenta satelite turismo argentina ( cst-a ) ',
 ' resto indicadores , buscan medir acceso turismo nivel desigualdad , toman fuente datos encuesta viajes turismo hogares ( evyth ) , representativa grandes aglomerados pais ',
 ' indicadores • puestos trabajo industrias turisticas : surge suma puestos trabajo existentes establecimientos pertenecientes ramas actividad clasificadas caracteristicas turismo ',
 ' incluye puestos trabajo asalariados ( registrados registrados ) asalariados ',
 ' • tasa informalidad empleo asalariado turistico : proporcion puestos trabaj

In [877]:
# Creamos una instancia de CountVectorizer
vectorizer1 = CountVectorizer()

In [878]:
# Ajustamos el vectorizador a nuestro corpus y transformamos nuestro corpus en vectores de conteo
X1 = vectorizer1.fit_transform(texto_plano)

# Imprimimos los vectores de características
print("Vectores de características:\n", X1.toarray())

# Imprimimos las palabras del vocabulario
print("\nPalabras del vocabulario:", "\n",
      vectorizer1.get_feature_names_out())

# Convertimos la matriz en un DataFrame de pandas para una mejor visualización
df1 = pd.DataFrame(X1.toarray(), columns=vectorizer1.get_feature_names_out())

# Imprimimos el DataFrame
print('\nVectores con palabras como columnas:')
print(df1)


Vectores de características:
 [[0 0 0 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 2 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

Palabras del vocabulario: 
 ['100' '12' 'aborda' 'acceso' 'actividad' 'aglomerados' 'amigos' 'ano'
 'aplicada' 'aporte' 'aquellos' 'argentina' 'asalariado' 'asalariados'
 'atribuible' 'buscan' 'cantidad' 'caracteristicas' 'caso' 'cercanos'
 'clasificadas' 'conceptos' 'conjunto' 'consumo' 'cst' 'cuenta' 'datos'
 'definiciones' 'dentro' 'desarrollo' 'describe' 'desigualdad' 'detalle'
 'directamente' 'distribucion' 'documento' 'economia' 'economica'
 'elaborados' 'empleo' 'encuesta' 'esparcimiento' 'establecimientos'
 'este' 'evyth' 'excluyendose' 'existentes' 'familiares' 'fuentas'
 'fuente' 'fuentes' 'gasto' 'gini' 'grandes' 'habitual' 'hogares'
 'igualdad' 'implican' 'incluye' 'indicador' 'indicadores' 'indice'
 'industrias' 'informacion' 'informalidad' 'ingresos' 'mayor' 'medida'
 'medir' 'menos' 'mientras' 'motivo' 'motivos' 'negocios'

In [879]:
texto = convertir_a_minusculas(texto)

In [880]:
texto = remueve_acentos(texto)

In [881]:
texto = remueve_palabras(texto)

In [882]:
texto = texto.split(".")

In [883]:
vectorizer2 = CountVectorizer()

In [884]:
X2 = vectorizer2.fit_transform(texto)

# Imprimimos los vectores de características
print("Vectores de características:\n", X2.toarray())

# Imprimimos las palabras del vocabulario
print("\nPalabras del vocabulario:", "\n",
      vectorizer2.get_feature_names_out())

# Convertimos la matriz en un DataFrame de pandas para una mejor visualización
df2 = pd.DataFrame(X2.toarray(), columns=vectorizer2.get_feature_names_out())

# Imprimimos el DataFrame
print('\nVectores con palabras como columnas:')
print(df2)


Vectores de características:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

Palabras del vocabulario: 
 ['10' '12' '13' '14' '15' '16' '17' '18' '180' '19' '25' '549' 'abonar'
 'academicas' 'accedan' 'acceder' 'acceso' 'actividades' 'acuerdos'
 'adecuadas' 'administracion' 'administrativas' 'administrativos' 'aereos'
 'afianzando' 'alojamiento' 'ambiental' 'analizar' 'ancianos' 'aplicacion'
 'aplican' 'aporte' 'aquellos' 'argentinos' 'articulo' 'artiwlo'
 'asesoramiento' 'asi' 'asignacion' 'asilos' 'asociaciones' 'auspicien'
 'autoridad' 'autorizaciones' 'ayuda' 'balnearios' 'bellezas'
 'beneficiarios' 'beneficio' 'brindara' 'cada' 'caducidad' 'campos'
 'capacidades' 'capitulo' 'carenciadas' 'caso' 'celebracion' 'ciento'
 'cinco' 'colaboracion' 'colonias' 'comida' 'comodidad' 'compatible'
 'complementarias' 'comprende' 'condiciones' 'conformacion' 'conocimiento'
 'consecuencia' 'consejo' 'conservando' 'contara' 

In [885]:
# Inicializamos el TfidfVectorizer
vectorizer_tfidf = TfidfVectorizer()

In [886]:
# Ajustamos y transformamos nuestro corpus
X_tfidf_1 = vectorizer_tfidf.fit_transform(texto_plano)

# Mostramos las características (palabras únicas en el corpus)
print("Características: ", vectorizer_tfidf.get_feature_names_out())

# Mostramos la matriz TF-IDF resultante
print("\nMatriz TF-IDF:")
print(X_tfidf_1.toarray())

# Resumen
print("\nVocabulario:")
print(vectorizer_tfidf.vocabulary_)
print("\nIDF:")
print(vectorizer_tfidf.idf_)

Características:  ['100' '12' 'aborda' 'acceso' 'actividad' 'aglomerados' 'amigos' 'ano'
 'aplicada' 'aporte' 'aquellos' 'argentina' 'asalariado' 'asalariados'
 'atribuible' 'buscan' 'cantidad' 'caracteristicas' 'caso' 'cercanos'
 'clasificadas' 'conceptos' 'conjunto' 'consumo' 'cst' 'cuenta' 'datos'
 'definiciones' 'dentro' 'desarrollo' 'describe' 'desigualdad' 'detalle'
 'directamente' 'distribucion' 'documento' 'economia' 'economica'
 'elaborados' 'empleo' 'encuesta' 'esparcimiento' 'establecimientos'
 'este' 'evyth' 'excluyendose' 'existentes' 'familiares' 'fuentas'
 'fuente' 'fuentes' 'gasto' 'gini' 'grandes' 'habitual' 'hogares'
 'igualdad' 'implican' 'incluye' 'indicador' 'indicadores' 'indice'
 'industrias' 'informacion' 'informalidad' 'ingresos' 'mayor' 'medida'
 'medir' 'menos' 'mientras' 'motivo' 'motivos' 'negocios' 'nivel'
 'objetivos' 'ocio' 'ods' 'pais' 'parte' 'pernocte' 'pertenecientes' 'pib'
 'poblacion' 'porcentaje' 'presenta' 'profesionales' 'propension'
 'proporcio

In [887]:
# Inicializamos el TfidfVectorizer
vectorizer2_tfidf = TfidfVectorizer()

In [888]:
# Ajustamos y transformamos nuestro corpus
X_tfidf_2 = vectorizer2_tfidf.fit_transform(texto_plano)

# Mostramos las características (palabras únicas en el corpus)
print("Características: ", vectorizer2_tfidf.get_feature_names_out())

# Mostramos la matriz TF-IDF resultante
print("\nMatriz TF-IDF:")
print(X_tfidf_2.toarray())

# Resumen
print("\nVocabulario:")
print(vectorizer2_tfidf.vocabulary_)
print("\nIDF:")
print(vectorizer2_tfidf.idf_)

Características:  ['100' '12' 'aborda' 'acceso' 'actividad' 'aglomerados' 'amigos' 'ano'
 'aplicada' 'aporte' 'aquellos' 'argentina' 'asalariado' 'asalariados'
 'atribuible' 'buscan' 'cantidad' 'caracteristicas' 'caso' 'cercanos'
 'clasificadas' 'conceptos' 'conjunto' 'consumo' 'cst' 'cuenta' 'datos'
 'definiciones' 'dentro' 'desarrollo' 'describe' 'desigualdad' 'detalle'
 'directamente' 'distribucion' 'documento' 'economia' 'economica'
 'elaborados' 'empleo' 'encuesta' 'esparcimiento' 'establecimientos'
 'este' 'evyth' 'excluyendose' 'existentes' 'familiares' 'fuentas'
 'fuente' 'fuentes' 'gasto' 'gini' 'grandes' 'habitual' 'hogares'
 'igualdad' 'implican' 'incluye' 'indicador' 'indicadores' 'indice'
 'industrias' 'informacion' 'informalidad' 'ingresos' 'mayor' 'medida'
 'medir' 'menos' 'mientras' 'motivo' 'motivos' 'negocios' 'nivel'
 'objetivos' 'ocio' 'ods' 'pais' 'parte' 'pernocte' 'pertenecientes' 'pib'
 'poblacion' 'porcentaje' 'presenta' 'profesionales' 'propension'
 'proporcio